In [2]:
import boto3
import sagemaker
import os
from sagemaker import get_execution_role

region = boto3.session.Session().region_name
print(region)

#role = get_execution_role()
role = 'arn:aws-cn:iam::690704700794:role/service-role/AmazonSageMaker-ExecutionRole-20200430T123312'

cn-northwest-1


We will demo using Sagemaker inference in BYOC mode, so first we need package our container.

We are using AWS Deep Learning Container as our base container, you can check the available list in https://aws.amazon.com/cn/releasenotes/available-deep-learning-containers-images/

Remember change the base container by the region you are using.

# Container build有两种方式（二选一）

*  自己构建（在中国区会较慢）

*   使用现有的(推荐)

### 本次任务可以使用已经封装的docker image

## 创建ECR 

In [3]:
# Run this cell only onece to create the repository in ECR
import boto3

account_id = boto3.client('sts').get_caller_identity().get('Account')
ecr_repository = 'ocr-inference-container'
tag = ':latest'
uri_suffix = 'amazonaws.com'
if region in ['cn-north-1', 'cn-northwest-1']:
    uri_suffix = 'amazonaws.com.cn'
inference_repository_uri = '{}.dkr.ecr.{}.{}/{}'.format(account_id, region, uri_suffix, ecr_repository + tag)
print(inference_repository_uri)
ecr = '{}.dkr.ecr.{}.{}'.format(account_id, region, uri_suffix)

!aws ecr create-repository --repository-name $ecr_repository

690704700794.dkr.ecr.cn-northwest-1.amazonaws.com.cn/ocr-inference-container:latest
{
    "repository": {
        "repositoryArn": "arn:aws-cn:ecr:cn-northwest-1:690704700794:repository/ocr-inference-container",
        "registryId": "690704700794",
        "repositoryName": "ocr-inference-container",
        "repositoryUri": "690704700794.dkr.ecr.cn-northwest-1.amazonaws.com.cn/ocr-inference-container",
        "createdAt": 1596479550.0
    }
}


## 1. 自己构建（在中国区会较慢）

In [5]:
inference_repository_uri

'690704700794.dkr.ecr.cn-northwest-1.amazonaws.com.cn/ocr-inference-container:latest'

### Build and push

In [6]:
!aws ecr get-login-password --region cn-northwest-1 | docker login --username AWS --password-stdin 727897471807.dkr.ecr.cn-northwest-1.amazonaws.com.cn

usage: aws [options] <command> <subcommand> [<subcommand> ...] [parameters]
To see help text, you can run:

  aws help
  aws <command> help
  aws <command> <subcommand> help
aws: error: argument operation: Invalid choice, valid choices are:

batch-check-layer-availability           | batch-delete-image                      
batch-get-image                          | complete-layer-upload                   
create-repository                        | delete-lifecycle-policy                 
delete-repository                        | delete-repository-policy                
describe-images                          | describe-repositories                   
get-authorization-token                  | get-download-url-for-layer              
get-lifecycle-policy                     | get-lifecycle-policy-preview            
get-repository-policy                    | initiate-layer-upload                   
list-images                              | put-image                               
pu

In [ ]:
!aws ecr get-login-password --region $region | docker login --username AWS --password-stdin $ecr

# Create ECR repository and push docker image
!docker build -t $ecr_repository ../recognition

!docker tag {ecr_repository + tag} $inference_repository_uri
!docker push $inference_repository_uri

## 2. 使用现有的docker镜像

将现有镜像下载到本地，并推送到自己的ECR库中

In [ ]:
# 不要改下面这行命令
!aws ecr get-login-password --region cn-north-1 | docker login --username AWS --password-stdin 346044390830.dkr.ecr.cn-north-1.amazonaws.com.cn

In [ ]:
exist_image = '346044390830.dkr.ecr.cn-north-1.amazonaws.com.cn/spoken-language-identification-sagemaker-inference-container:latest'

In [ ]:
!docker pull $exist_image

In [ ]:
!aws ecr get-login-password --region $region | docker login --username AWS --password-stdin $ecr

In [ ]:
!docker tag $exist_image $inference_repository_uri
!docker push $inference_repository_uri

## Inference（两种方式均可）

In [ ]:
inference_repository_uri

### 注意

**将如下model_uri改为在training阶段得到的模型在S3中的path，形式为s3://YOUR_BUCKET_NAME/spoken/output/tensorflow-training-x-x-x-x-x-x-x/output/model.tar.gz**， 可以在console找到该训练任务，在该训练任务的描述页面中，找到“S3 模型构件”，复制即可。

In [ ]:
image = inference_repository_uri
# update model_uri to your model S3 uri
model_uri = 'YOUR_MODEL_URI'



推理请求的结构是发送一个json结构体，json结构体里面描述：

bucket: 存放待推理音频数据的存储桶

audio_uri:待推理音频数据在S3的uri，不含桶名

class_count: 语音语言种类，与模型训练时强相关，即模型训练的时候提供了几种语言的种类，这儿就填几，如训练时提供了5种语言，这里就写5；

**即发送推理请求前，先将待推理的音频文件上传到S3**

## 测试文件格式说明

bucket为保存待推理的音频文件桶名，audio_uri为该待推理文件在S3中的uri，且不含有桶名，即只有前缀，如一个名为demo1.mp3文件上传到桶名为test的s3存储桶后（且audio目录下），
则该文件的S3 uri为s3://test/audio/demo1.mp3, 其中test为桶名，audio/demo1.mp3即为下面audio_uri的值；


```

#s3://test_bucket/audio/demo1.mp3

bucket = 'test_bucket'
audio_uri = 'audio/demo1.mp3'

```


In [ ]:
import json
# bucket为保存待推理的音频文件桶名，audio_uri为该待推理文件在S3中的uri，且不含有桶名，即只有前缀，如一个名为demo1.mp3文件上传到桶名为test的s3存储桶后（且audio目录下），
# 则该文件的S3 uri为s3://test/audio/demo1.mp3, 其中test为桶名，audio/demo1.mp3即为下面audio_uri的值；
bucket = 'YOUR_BUCKET_SOTRE_AUDIO_TO_INFERENCE'
audio_uri = 'xxxxxxxx.mp3'


test_data = {
    'bucket' : bucket,
    'audio_uri' : audio_uri,
    'class_count' : '3'
}
payload = json.dumps(test_data)


In [ ]:
print(payload)

### Method 1: Using sagemaker SDK

In [ ]:
# Below could be modified as you want

initial_instance_count = 1
instance_type = 'ml.m5.large'
endpoint_name= 'spl-endpoint-July'

In [ ]:
# 创建 model

from sagemaker.model import Model
image = inference_repository_uri
tfModel =Model(
            model_data=model_uri, 
            role=role,
            image=image)

In [ ]:
# 创建 endpoint

tfModel.deploy(
    initial_instance_count=initial_instance_count,
    instance_type=instance_type,
    endpoint_name=endpoint_name)

In [ ]:
# 创建推理用的 predictor

new_predictor = sagemaker.predictor.RealTimePredictor(
    endpoint=endpoint_name,
    content_type='application/json')

In [ ]:
# 推理请求代码

new_sm_response = new_predictor.predict(payload)

print(json.loads(new_sm_response.decode()))

### Method 2: Using boto3 SDK

In [ ]:
# Below could be modified as you want

model_name = 'spl-demo'
endpoint_config_name='spl-endpoint-config-July'
variant_name= 'spl-variant-1-June'
initial_instance_count = 1
instance_type = 'ml.m5.large'
endpoint_name= 'spl-endpoint-July'

In [ ]:
import boto3

sm_client = boto3.client('sagemaker')

# create model object

spl_model_demo = sm_client.create_model(
    ModelName=model_name,
    PrimaryContainer={
        'Image': image,
        'Mode': 'SingleModel',
        'ModelDataUrl': model_uri,
    },
    ExecutionRoleArn= role, 
    EnableNetworkIsolation=False
)

In [ ]:
# create endpoint config

spl_endpoint_config = sm_client.create_endpoint_config(
    EndpointConfigName=endpoint_config_name,
    ProductionVariants=[
        {
            'VariantName': variant_name,
            'ModelName': model_name,
            'InitialInstanceCount': initial_instance_count,
            'InstanceType': instance_type
        },
    ]
)

In [ ]:
# create endpoint

response = sm_client.create_endpoint(
    EndpointName=endpoint_name,
    EndpointConfigName=endpoint_config_name
)

待上一步创建完成后再进行下面的发送推理请求。上面创建endpoint的时间大概10分钟左右，可以在console查看状态，inservice即可使用了。

推理代码：

In [ ]:
import boto3
import json
import time

region_name='cn-northwest-1'
profile_name='default'

session = boto3.session.Session(region_name=region_name, profile_name=profile_name)
client = session.client('sagemaker-runtime')

start_time = time.time()
spl_response=client.invoke_endpoint(EndpointName=endpoint_name,
        Body=payload,
        ContentType='application/json')
end_time = time.time()

print('time cost %s s' %(end_time - start_time))
print(json.loads(spl_response['Body'].read().decode()))